In [1]:
import torch
from torch import nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.randn(size=(2,4),requires_grad=True)
net(X)

tensor([[ 0.4685],
        [-0.2129]], grad_fn=<AddmmBackward0>)

## 5.2.1参数访问

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1755, -0.2414, -0.1204,  0.2782,  0.2008, -0.0312, -0.0421, -0.2037]])), ('bias', tensor([-0.1122]))])


### 5.2.1.1目标参数

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1122], requires_grad=True)
tensor([-0.1122])


In [4]:
net[2].weight.grad==None

True

### 5.2.1.2一次性访问所有参数

In [5]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([-0.1122])

### 5.2.1.3从嵌套块收集参数

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                         nn.Linear(8,4),nn.ReLU())

def block2():
    net=nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block{i}',block1())
    return net

rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.4887],
        [0.4887]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.0644, -0.4073, -0.1460,  0.3281, -0.1280, -0.1128,  0.0769, -0.0689])

## 5.2.2参数初始化
### 5.2.2.1内置初始化

In [10]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0044, -0.0086, -0.0060, -0.0027]), tensor(0.))

In [11]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [12]:
def init_xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)
        
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.3575, 0.3898, 0.4535, 0.2670])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 5.2.2.2自定义初始化

In [13]:
def my_init(m):
    if type(m)==nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *=m.weight.data.abs()  >=5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  9.6827,  0.0000,  7.7525],
        [-0.0000,  9.0438,  0.0000, -7.9825]], grad_fn=<SliceBackward0>)

In [14]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000, 10.6827,  1.0000,  8.7525])

## 5.2.3参数绑定

In [16]:
# 我们需要给共享层提供一个名称，以便可以引用它的参数
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                  shared,nn.ReLU(),
                  shared,nn.ReLU(),
                  nn.Linear(8,1))

net(X)
# 检查参数是否相同
print(net[2].weight.data[0]==net[4].weight.data[0])
net[2].weight.data.data[0,0]=100
# 确保它们实际上是同一个对象，而不是只用相同的值
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
